In [7]:
# Load modules, mainly huggingface basic model handlers.
# Make sure you install huggingface and other packages properly.
from collections import Counter
import json
import copy
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import matthews_corrcoef

import logging
logger = logging.getLogger(__name__)

import os
os.environ["TRANSFORMERS_CACHE"] = "../huggingface_cache/" # Not overload common dir 
                                                           # if run in shared resources.

import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import torch
import argparse
import numpy as np
import pandas as pd
from datasets import load_dataset, load_metric
from datasets import Dataset
from datasets import DatasetDict

import transformers
from transformers import (
    AutoModelForMaskedLM,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
    EarlyStoppingCallback
)
from transformers.trainer_utils import is_main_process, EvaluationStrategy
from functools import partial

In [8]:
# this file contains data preprocessor for the following GLUE datasets.
# CoLA(!), SST(!), MRPC(!), STS-B, QQP, MNLI-m, MNLI-mm, QNLI(!), RTE, WNLI

# the following dataset are not preprocessed yet.
# STS-B, QQP, MNLI-m, MNLI-mm, RTE, WNLI

In [77]:
glue_task = "qqp"

In [78]:
dataset = load_dataset(
    "glue", glue_task,
    cache_dir="../../huggingface_cache/"
)

In [79]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'question1', 'question2'],
        num_rows: 363846
    })
    validation: Dataset({
        features: ['idx', 'label', 'question1', 'question2'],
        num_rows: 40430
    })
    test: Dataset({
        features: ['idx', 'label', 'question1', 'question2'],
        num_rows: 390965
    })
})

In [74]:
dataset["train"][100]

{'idx': 114,
 'label': 0,
 'sentence1': 'The Nasdaq composite index inched up 1.28 , or 0.1 percent , to 1,766.60 , following a weekly win of 3.7 percent .',
 'sentence2': 'The technology-laced Nasdaq Composite Index .IXIC was off 24.44 points , or 1.39 percent , at 1,739.87 .'}

In [75]:
dataset["test"][100]

{'idx': 100,
 'label': 1,
 'sentence1': 'Licensing revenue slid 21 percent , however , to $ 107.6 million .',
 'sentence2': 'License sales , a key measure of demand , fell 21 percent to $ 107.6 million .'}

In [76]:
dataset.save_to_disk(f"../../data-files/{glue_task}")